### SVM 

Support Vector Machine (SVM) is a powerful classification machine learning algorithm.

Considering samples $D=\{(x_1, y_1), (x_2, y_2),..., (x_m, y_m)\}$,
Sample space can be divided by such hyperplane $w
^Tx+b=0$ where $w = (w_1; w_2; ...; w_d)$ is the normal vector, which decides the direction of the hyperplane, and $b$ is the intercept. 

The distance between any point $x$ in the sample space to the hyperplane is
$$r = \frac{|w^Tx+b|}{||w||}$$


Suppose the hyperplane $(w,b)$ can classify the training samples correctly, i.e. for $(x_i,y_i) \in D$, if $y_i=+1$ then we have $w^Tx_i+b > 0$. If $y_i=-1$ then $w^Tx_i +b <0$

Let $$\left\{
         \begin{array}{ll}
           w^Tx_i+b \geq +1, & y_i=+1\\
           w^Tx_i+b \leq -1, & y_i=-1
                \end{array}
              \right.$$